# A. Process data and fill in missing values

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("adj_close_hist.csv")
print("Total rows: {0}".format(len(data)))
print(list(data))

Total rows: 1793
['date', 'A', 'AMZN', 'CAT', 'GDP', 'MSFT', 'NFLX', 'TSLA', 'XOM']


In [2]:
data.fillna(0)
filled_data = data.fillna(method='pad')

In [3]:
arr_data = np.array(filled_data)
arr_data

array([['1/3/2011', 28.57223993, 184.22, ..., 25.48714286, 26.62,
        60.4157331],
       ['1/4/2011', 28.30616607, 185.01, ..., 25.91, 26.67, 60.69937504],
       ['1/5/2011', 28.244764399999998, 187.42, ..., 25.67571429, 26.83,
        60.53729393],
       ...,
       ['2/14/2018', 69.7, 1451.05, ..., 266.0, 322.31, 76.46],
       ['2/15/2018', 72.02, 1461.76, ..., 280.27, 334.065, 76.21],
       ['2/16/2018', 71.97, 1448.69, ..., 278.52, 335.49, 76.54]],
      dtype=object)

# B. Implement trading strategies

In [4]:
#calculating SMA
def SMA(x,n,col):
    avgs = []
    for i in range(len(x)-n+1):
            avgs.append(sum(x[i:i+n,col])/n)
    return avgs

#print(len(SMA(arr_data,5,1)))

#initial position: long
#getting position cash flows
def position_cf(x,n,N,col):
    position = []
    SMA_n = SMA(x,n,col)[N-n:]
    SMA_N = SMA(x,N,col)

    #comparing shor-term and long-term SMA
    for i in range(len(x)-N+1):
        if SMA_n[i] > SMA_N[i]:
            position.append(-x[i+N-1,col])
        if SMA_n[i] < SMA_N[i]:
            position.append(x[i+N-1,col])
        elif SMA_n[i] == SMA_N[i]:
            position.append(position[i-1])
    return position

position_cf(arr_data,1,5,1)

[-28.39485735,
 -28.80420177,
 -29.29541506,
 -29.42504079,
 -29.31588228,
 -29.51373208,
 -30.25737443,
 28.94747231,
 28.85195861,
 28.72915529,
 29.17943414,
 -29.12485489,
 -29.04298601,
 28.96111712,
 27.95822332,
 28.5381279,
 -28.68822085,
 28.12878349,
 27.96504572,
 -29.32952709,
 -30.31877609,
 -30.134571100000002,
 -29.629713,
 -30.01859019,
 -30.71447569,
 -30.55756033,
 29.09756526,
 29.72522669,
 29.96401093,
 -29.96401093,
 28.98840675,
 27.59663575,
 28.07420423,
 -28.89971546,
 -28.70868807,
 27.75355111,
 28.28569885,
 -29.21354618,
 -31.89475207,
 -31.29438027,
 -31.36260434,
 -31.41718359,
 30.61896199,
 30.93279271,
 30.59167236,
 29.89578686,
 28.87924824,
 28.96111712,
 29.29541506,
 -29.98447815,
 -29.46597523,
 -29.79345076,
 -29.95718853,
 -30.04587982,
 -29.99130056,
 -30.40064497,
 -30.76223254,
 -30.55073792,
 -30.79634457,
 30.58484996,
 -30.93279271,
 -30.89868067,
 30.61213958,
 30.625784399999997,
 30.69400847,
 30.65989643,
 -31.78559356,
 -31.81288319

In [5]:
#getting transaction cash flows
def transaction_cf(x,n,N,col):
    tran=[]
    p = position_cf(arr_data,n,N,col)
    tran.append(p[0])
    #Buy and hold until SMA_n < SMA_N and then sell it  
    for j in range(0,len(p)):
        if ((p[j]*p[j-1])<0) & (p[j-1]<0):
            tran.append(p[j])
        if ((p[j]*p[j-1])<0) & (p[j-1]>0):
            tran.append(p[j])
    return tran

transaction_cf(arr_data,1,5,1)

[-28.39485735,
 28.94747231,
 -29.12485489,
 28.96111712,
 -28.68822085,
 28.12878349,
 -29.32952709,
 29.09756526,
 -29.96401093,
 28.98840675,
 -28.89971546,
 27.75355111,
 -29.21354618,
 30.61896199,
 -29.98447815,
 30.58484996,
 -30.93279271,
 30.61213958,
 -31.78559356,
 31.88792967,
 -33.18418697,
 33.77091396,
 -34.03016542,
 34.91025591,
 -35.29231069,
 34.903433500000006,
 -33.69586749,
 32.74755293,
 -32.54288073,
 32.42689981,
 -33.4093264,
 32.3177413,
 -33.00680439,
 33.08185087,
 -34.43950983,
 34.26894966,
 -32.01073299,
 31.65596783,
 -23.54412608,
 24.42421656,
 -22.568521899999997,
 24.45150619,
 -24.34917009,
 23.29169702,
 -23.72833106,
 23.85113439,
 -22.89599743,
 21.58609531,
 -22.18646711,
 21.32002144,
 -21.91357084,
 21.40189032,
 -22.72543725,
 22.15235508,
 -22.84141817,
 22.50712023,
 -23.01197834,
 25.29066223,
 -25.18150372,
 24.60842155,
 -25.556736100000002,
 25.38617593,
 -23.98758252,
 25.22926057,
 -22.82777336,
 22.670858,
 -23.59188292,
 23.2712298

# C. Comapre performance statistics for the trading strategies

# a. Average position turn-over per annum

In [6]:
import datetime as dt
from datetime import timedelta

def avg_turnover_per_ann(x,n,N,col):
    date = []
    nyears = 0
    tran_num = 0
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))
        
    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
    tran_num = len(transaction_cf(x,n,N,col))/2
    return tran_num/nyears

In [7]:
avg_turnover_per_ann(arr_data,1,5,1)

33.46885813150353

# b. Average position holding period

In [13]:
def avg_hp(x,n,N,col):
    trandate_index=[]
    p = position_cf(x,n,N,col)
    trandate_index=[]
    trandate=[]
    hp_sum=0
    trandate_index.append(N-1)
    date = []
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))
        
    #Buy and hold until SMA_n < SMA_N and then sell it  
    for j in range(0,len(p)):
        if ((p[j]*p[j-1])<0) & (p[j-1]<0):
            trandate_index.append(j+N-1)
        if ((p[j]*p[j-1])<0) & (p[j-1]>0):
            trandate_index.append(j+N-1)
            
    for i in range(0,len(trandate_index)-1,2):
        hp_sum += (date[trandate_index[i+1]]-date[trandate_index[i]]).days        

    return hp_sum/(len(transaction_cf(x,n,N,col))/2)

avg_hp(arr_data,1,5,1)

6.20125786163522

# c. Average annualized return

In [22]:
def avg_ann_ret(x,n,N,col,principal):
    shares = 0
    profit = 0
    rets = []
    total_ret = 1
    date = []
    nyears = 0
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))

    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
       
    a = transaction_cf(x,n,N,col)
    for i in range(0,len(a)-1,2):
        shares = principal/(abs(a[i]))
        profit = (a[i+1] + a[i])*shares
        rets.append(profit/principal)
        principal += profit

    for i in range(len(rets)):
        total_ret *= (rets[i]+1)
    
    return total_ret**(1/nyears)-1

avg_ann_ret(arr_data,1,5,1,10000)

0.02038624040820869

# d. Average annualized return volatility

In [15]:
def avg_ann_ret_vol(x,n,N,col,principal):
    shares = 0
    profit = 0
    rets = []
    a = transaction_cf(x,n,N,col)
    date = []
    nyears = 0
    
    for i in arr_data[:,0]:
        date.append(dt.datetime.strptime(i,'%m/%d/%Y'))

    y = ((date[-1]-date[0])/365)
    nyears = y.total_seconds() / timedelta(days=1).total_seconds()
    
    for i in range(0,len(a)-1,2):
        shares = principal/(abs(a[i]))
        profit = (a[i+1] + a[i])*shares
        rets.append(profit/principal)
        principal += profit
        
    return np.var(rets)/nyears

avg_ann_ret_vol(arr_data,1,5,1,10000)

0.00012569067029363396

# Summary: Outputs for all stocks

In [28]:
for i in range(1,arr_data.shape[1]):
    print('====================',list(data)[i],'====================')
    print('Short-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,1,5,i))
    print('Medium-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,15,50,i))
    print('Long-term average position turn-over per annum:',avg_turnover_per_ann(arr_data,50,200,i),'\n')
    
    print('Short-term average position holding period:',avg_hp(arr_data,1,5,i))
    print('Medium-term average position holding period:',avg_hp(arr_data,15,50,i))
    print('Long-term average position holding period:',avg_hp(arr_data,50,200,i),'\n')
    
    print('Short-term average annualized return:',avg_ann_ret(arr_data,1,5,i,10000))
    print('Medium-term average annualized return:',avg_ann_ret(arr_data,15,50,i,10000))
    print('Long-term average annualized return:',avg_ann_ret(arr_data,50,200,i,10000),'\n')
    
    print('Short-term average annualized return volatility:',avg_ann_ret_vol(arr_data,1,5,i,10000))
    print('Medium-term average annualized return volatility:',avg_ann_ret_vol(arr_data,15,50,i,10000))
    print('Long-term average annualized return volatility:',avg_ann_ret_vol(arr_data,50,200,i,10000),'\n')

==================== A ====================
Short-term average position turn-over per annum: 33.46885813150353
Medium-term average position turn-over per annum: 2.8767781622466346
Long-term average position turn-over per annum: 0.9121491733952743 

Short-term average position holding period: 6.20125786163522
Medium-term average position holding period: 75.3170731707317
Long-term average position holding period: 151.23076923076923 

Short-term average annualized return: 0.02038624040820869
Medium-term average annualized return: 0.08213912760527031
Long-term average annualized return: 0.13488182442902796 

Short-term average annualized return volatility: 0.00012569067029363396
Medium-term average annualized return volatility: 0.0016807310036209768
Long-term average annualized return volatility: 0.08238262637738744 

==================== AMZN ====================
Short-term average position turn-over per annum: 32.90753556326028
Medium-term average position turn-over per annum: 2.59611687